In [2]:
'''
Source: https://medium.datadriveninvestor.com/scraping-live-stock-fundamental-ratios-news-and-more-with-python-a716329e0493

'''

'\nSource: https://medium.datadriveninvestor.com/scraping-live-stock-fundamental-ratios-news-and-more-with-python-a716329e0493\n\n'

In [162]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen, Request

In [163]:
# Input symbol
symbol = 'TSLA'

# Set up scraper
url = ("http://finviz.com/quote.ashx?t=" + symbol.lower())
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
html = soup(webpage, "html.parser")

In [214]:
# Locate fundamentals table
fundamentals = pd.read_html(str(html), attrs = {'class': 'snapshot-table2'})[0]
fundamentals.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,Index,S&P 500,P/E,1116.40,EPS (ttm),0.62,Insider Own,0.10%,Shs Outstand,959.85M,Perf Week,-6.68%
1,Market Cap,666.52B,Forward P/E,117.36,EPS next Y,5.92,Insider Trans,-33.54%,Shs Float,770.38M,Perf Month,13.60%
2,Income,690.00M,PEG,31.27,EPS next Q,0.79,Inst Own,46.00%,Short Float,6.64%,Perf Quarter,-19.64%
3,Sales,31.54B,P/S,21.14,EPS this Y,165.00%,Inst Trans,0.10%,Short Ratio,1.50,Perf Half Y,65.22%
4,Book/sh,23.37,P/B,29.71,EPS next Y,36.97%,ROA,1.60%,Target Price,650.81,Perf Year,351.43%


In [215]:
# Reshape dataframe to get two columns dataframe and change column names
columns = fundamentals.values.reshape(-1,2)
fundamentals = pd.DataFrame(columns)
fundamentals['Fundamentals'] = fundamentals[0]
fundamentals['Values'] = fundamentals[1]
fundamentals = fundamentals.drop(fundamentals.columns[[0,1]], axis=1)
fundamentals

,Fundamentals,Values
0,Index,S&P 500
1,P/E,1116.40
2,EPS (ttm),0.62
3,Insider Own,0.10%
4,Shs Outstand,959.85M
...,...,...
67,SMA20,-1.68%
68,SMA50,0.34%
69,SMA200,23.65%
70,Volume,21795135


In [201]:
# Locate news table, change column names and set date as index
news = pd.read_html(str(html), attrs = {'class': 'fullview-news-outer'})[0]
news['Date'] = news[0]
news['Title'] = news[1]
news = news.drop(news.columns[[0,1]], axis=1)
news = news.set_index('Date')
news

,Title
Date,
Apr-28-21 10:20PM,Dow Jones Futures Extend Gains As Biden Touts ...
09:02PM,Teslas Earnings Note a New Risk Barrons.com
08:53PM,ARK Invest Stocks To Buy And Watch: 6 Stocks T...
08:11PM,"Auto Chip Shortage Crisis Worsens With BMW, Fo..."
06:53PM,UPDATE 1-U.S. safety board to release report o...
...,...
04:12PM,"Tesla edges past Wall Street revenue target, b..."
04:10PM,"Tesla posts Q1 earnings that top estimates, do..."
04:05PM,Tesla Releases First Quarter 2021 Financial Re...


In [248]:
# Locate insider trading table, and set the first row as column title
insider_trading = pd.read_html(str(html), attrs = {'class':'body-table'})[0]
insider_trading.columns = insider_trading.iloc[0]
insider_trading = insider_trading.drop([0], axis=0)
insider_trading.head()

,Insider Trading,Relationship,Date,Transaction,Cost,#Shares,Value ($),#Shares Total,SEC Form 4
1,DENHOLM ROBYN M,Director,Apr 26,Option Exercise,52.38,62500,3273750,67500,Apr 28 09:41 PM
2,DENHOLM ROBYN M,Director,Apr 26,Sale,740.97,62500,46310388,5000,Apr 28 09:41 PM
3,Kirkhorn Zachary,Chief Financial Officer,Apr 19,Sale,718.00,1250,897500,55984,Apr 21 08:23 PM
4,Guillen Jerome M,"President, Heavy Trucking",Apr 12,Option Exercise,55.32,10000,553200,60598,Apr 14 07:13 PM
5,Guillen Jerome M,"President, Heavy Trucking",Apr 12,Sale,697.87,10000,6978659,50598,Apr 14 07:13 PM
